In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.data import Field

from dataset.mtgcards import RuleText
from utils.preprocess import fields_for_rule_text

import random
from tqdm import tqdm

import models.model6_1 as model6_1
from dataset.mtgcards import TestSets
from models.card_name_detector.definition import TrainedDetector
from utils.translate import sentencize, CardTranslator, CTHelper

d:\Miniconda3\envs\seq2seq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fields = {'src-rule': ('src', Field(tokenize=lambda x: x.split(' '))), 'trg-rule': ('trg', Field())}
test_data = TestSets.load(fields,'bro')

In [4]:
T = {
    'model4-T-v2.1': torch.load('result/model4-T-v2.1.pt'),
    'model4-T-v2.2': torch.load('result/model4-T-v2.2.pt'),
    #'model6-T-v2.1': torch.load('result/model6-T-v2.1.pt'),
    'model6-T-v2.2': torch.load('result/model6-T-v2.2.pt'),
    'model6.1-T-v2.2': torch.load('result/model6.1-T-v2.2.pt'),
    #'hybrid-T-v2.2': torch.load('result/hybrid-T-v2.2.pt'),
}
D = TrainedDetector()

path: d:\Desktop\mtg-cards-translation\models\card_name_detector


In [8]:
dic = {}
dic = {'oil':'烁油', 'rebel':'反抗军','compleat':'完化'}
helper = CTHelper(D, dic)
silent = True
CT={}
for key in T:
    value = T[key]
    CT[key]=CardTranslator(sentencize, value, 
                    preprocess=lambda x: helper.preprocess(x, silent), 
                    postprocess=lambda x: helper.postprocess(x, silent))

# ret = CT.translate(' '.join(example.src))
# print(ret)
from utils import calculate_testset_bleu
for example in random.sample(list(test_data), 1):
    for key in CT:
        ct=CT[key]
        print(vars(example))
        ret = ct.translate(' '.join(example.src))
        print(ret)
        calculate_testset_bleu(list(test_data)[:100], ct)



{'src': ['Flying\nArchfiend', 'of', 'the', 'Dross', 'enters', 'the', 'battlefield', 'with', 'four', 'oil', 'counters', 'on', 'it.\nAt', 'the', 'beginning', 'of', 'your', 'upkeep,', 'remove', 'an', 'oil', 'counter', 'from', 'Archfiend', 'of', 'the', 'Dross.', 'Then', 'if', 'it', 'has', 'no', 'oil', 'counters', 'on', 'it,', 'you', 'lose', 'the', 'game.\nWhenever', 'a', 'creature', 'an', 'opponent', 'controls', 'dies,', 'its', 'controller', 'loses', '2', 'life.'], 'trg': ['飞行', '深沼翼邪鬼进战场时上面有四个烁油指示物。', '在你的维持开始时，从深沼翼邪鬼上移去一个烁油指示物。然后如果其上没有烁油指示物，则你输掉这盘游戏。', '每当一个由对手操控的生物死去时，其操控者失去2点生命。']}
飞行 <archfiend of the dross>进战场时上面有四个充电指示物。 在你的维持开始时，从<archfiend of the dross>上放置一个烁油指示物。 然后如果其上有烁油上，则你输掉这盘游戏。 每当一个由对手操控的生物死去时，其操控者失去2点生命。
{'src': ['Flying\nArchfiend', 'of', 'the', 'Dross', 'enters', 'the', 'battlefield', 'with', 'four', 'oil', 'counters', 'on', 'it.\nAt', 'the', 'beginning', 'of', 'your', 'upkeep,', 'remove', 'an', 'oil', 'counter', 'from', 'Archfiend', 'of', 'the', 'Dross.', 'Then', 'if', 